# Seção 4: Classificação binária - base breast cancer
Os dados utilizados neste notebook foram tirados da página [Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29).

## Importação das bibliotecas

In [155]:
from random import randint
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

## Importação das bases

In [156]:
df_inputs = pd.read_csv("./data/entradas_breast.csv")
df_outputs = pd.read_csv("./data/saidas_breast.csv")

print(f"""- Formato do dataframe df_inputs: {df_inputs.shape}
- Formato do dataframe df_outputs: {df_outputs.shape}""")

- Formato do dataframe df_inputs: (569, 30)
- Formato do dataframe df_outputs: (569, 1)


### Análise das bases importadas:

In [157]:
# df_description = pd.DataFrame()
# for feature in df_inputs.columns:

### Pre-processamento

In [158]:
X_train, X_test, y_train, y_test = train_test_split(df_inputs, df_outputs, test_size=.2)

print(f"""{X_train.shape} e {y_train.shape}
{X_test.shape} e {y_test.shape}""")

(455, 30) e (455, 1)
(114, 30) e (114, 1)


### Modelo

In [159]:
def model_ann():
    """
    Nessa função está a criação da rede neural com a camada de entrada (incluida na adição da primeira camada intermediária),
    três camadas intermediárias e a camada de saída.
    Como trata-se de um problema de classificação binária, a função de ativação usada na camada de saída é a sigmóide.
    """
    model = Sequential()

    # Camada de entrada e intermediária
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))

    # Camada intermediária ou oculta
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))

    # Camada de saída
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Método de otimização a ser utilizado
    optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model

model = KerasClassifier(build_fn=model_ann, epochs=100, batch_size=20, verbose=0)

model.fit(X_train, y_train);

In [160]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred>.5,1,0)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.86      0.74        35
           1       0.93      0.80      0.86        79

    accuracy                           0.82       114
   macro avg       0.79      0.83      0.80       114
weighted avg       0.84      0.82      0.82       114

[[30  5]
 [16 63]]


## Usando validação cruzada

In [161]:
model = KerasClassifier(build_fn=model_ann, epochs=100, batch_size=20)

cross_val = cross_val_score(estimator=model, X=df_inputs, y=df_outputs
                            , cv=2, scoring="f1", verbose=0, n_jobs=-1)

cross_val.mean()

0.8560593220338983

# Overfitting e dropout
Para reduzir o risco de overfitting, pode-se usar o dropout. Esse método zera uma parcelas das entradas de camadas específicas.

In [164]:
def model_ann():
    """
    Nessa função está a criação da rede neural com a camada de entrada (incluida na adição da primeira camada intermediária),
    três camadas intermediárias e a camada de saída.
    Como trata-se de um problema de classificação binária, a função de ativação usada na camada de saída é a sigmóide.
    """
    model = Sequential()

    # Camada de entrada e intermediária
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
    model.add(Dropout(0.2))

    # Camada intermediária ou oculta
    model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))

    # Camada de saída
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Método de otimização a ser utilizado
    optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.25)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model

model = KerasClassifier(build_fn=model_ann, epochs=100, batch_size=20, verbose=0)

model.fit(X_train, y_train);

In [165]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred>.5,1,0)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.83        35
           1       0.94      0.91      0.92        79

    accuracy                           0.89       114
   macro avg       0.87      0.88      0.88       114
weighted avg       0.90      0.89      0.90       114

[[30  5]
 [ 7 72]]


## Tuning de hiperparâmetros
Vamos criar uma função baseada em uma criada anteriormente, mas dessa vez generalizaremos alguns dos hiperparâmetros.

In [63]:
def model_ann_tunned(optimizer, loss, kernel_initializer, activation, neurons):
    """
    Nessa função está a criação da rede neural com a camada de entrada (incluida na adição da primeira camada intermediária),
    três camadas intermediárias e a camada de saída.
    Como trata-se de um problema de classificação binária, a função de ativação usada na camada de saída é a sigmóide.
    
    Parâmetros:
        optimizer, loss, kernel_initializer, activation, neurons
    """
    model = Sequential()

    # Camada de entrada e intermediária
    model.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer, input_dim=30))
    model.add(Dropout(0.2))

    # Camada intermediária ou oculta
    model.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer))

    # Camada de saída
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Método de otimização a ser utilizado
    model.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])
    
    return model

model = KerasClassifier(build_fn=model_ann_tunned)

parametros = {'batch_size': [10, 30],
              'epochs': [50, 100],
              'optimizer': ['adam', 'sgd'],
              'loss': ['binary_crossentropy', 'hinge'],
              'kernel_initializer': ['random_uniform', 'normal'],
              'activation': ['relu', 'tanh'],
              'neurons': [16, 8]}

grid_search = GridSearchCV(estimator = model,
                           param_grid = parametros,
                           scoring = 'accuracy',
                           cv = 3)

grid_search = grid_search.fit(X_train, y_train)

melhores_parametros = grid_search.best_params_

melhor_precisao = grid_search.best_score_

## Previsão

In [ ]:
model = model_ann_tunned(melhores_parametros)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## Salvando o modelo
Abaixo um exemplo de como salvar os hiperparâmetros utilizados e os pesos obtidos no treinamento do modelo.

In [76]:
model = Sequential()

# Camada de entrada e intermediária
model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
model.add(Dropout(0.2))

# Camada intermediária ou oculta
model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))

# Camada de saída
model.add(Dense(units=1, activation='sigmoid'))

# Método de otimização a ser utilizado
optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

model_json = model.to_json()

with open("./model/model_ann_tunned.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("./model/model_ann_tunned.h5")

## Carregando o modelo

In [124]:
with open("./model/model_ann_tunned.json", "r") as file:
    loaded_model = file.read()

model = model_from_json(loaded_model)

model.load_weights("./model/model_ann_tunned.h5")

registro_teste = X_test[1:2]

y_pred = model.predict(registro_teste)
y_pred

array([[0.98901486]], dtype=float32)

## Exercício para casa
Nas aulas anteriores, chegamos no percentual de 84% de acerto utilizando validação cruzada juntamente com o uso de dropout . Com base nos parâmetros da rede neural que você aprendeu neste módulo, o objetivo é criar um modelo que ultrapasse a precisão de 90%. Para isso, você pode realizar alguns testes com base nos itens abaixo:

Deixar a rede neural mais profunda, adicionando mais camadas ocultas (nas aulas nós fizemos os testes com somente duas camadas)
Quando fizemos o tuning dos parâmetros chegamos a conclusão que os melhores resultados são obtidos se utilizarmos 8 neurônios nas camadas ocultas. Se aumentarmos essa quantidade para 32 será que teremos melhores resultados? E se colocarmos quantidade de neurônios diferentes nas camadas, como por exemplo: 16 neurônios na primeira camada e 8 na segunda?
Aumentar e/ou diminuir o percentual de neurônios zerados com o dropout  (lembre-se de que quanto maior o percentual, mais a rede neural tem a tendência a entrar no problema do underfitting)
Testar outras funções de ativação para as camadas ocultas, conforme as mostradas aqui na documentação do Keras
Testar outras formas para a inicialização dos pesos, conforme mostradas aqui na documentação do Keras
Testar outros otimizadores, conforme mostrados aqui na documentação do Keras. Após escolher o melhor otimizador, testar a configuração dos parâmetros learning rate , decay  e clipvalue 
Aumentar o número de épocas, bem como testas outros valores para o batch_size 
Para esta tarefa você pode utilizar como base o arquivo breast_cancer_cruzada.py  e utilizar o valor da variável media  para analisar o resultado da precisão.

Assim que você conseguir o resultado de 90%, você deve postar as configurações utilizadas!

In [154]:
model = Sequential()

# Camada de entrada e intermediária
model.add(Dense(units=8, activation='relu', kernel_initializer='random_uniform', input_dim=30))
model.add(Dropout(0.1))

# Camada intermediária ou oculta
model.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))

# Camada de saída
model.add(Dense(units=1, activation='sigmoid'))

# Método de otimização a ser utilizado
optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.25)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

X_train, X_test, y_train, y_test = train_test_split(df_inputs, df_outputs, test_size=.2)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred = np.where(y_pred>.5,1,0)

print(classification_report(y_test, y_pred))


15/15 [==============================] - 0s 1ms/step - loss: 1.6107 - binary_accuracy: 0.4923
              precision    recall  f1-score   support

           0       0.58      0.47      0.52        45
           1       0.69      0.78      0.73        69

    accuracy                           0.66       114
   macro avg       0.64      0.62      0.63       114
weighted avg       0.65      0.66      0.65       114

